In [ ]:
# read in necessary libraries, and specify the top level directory where
# all of the social distancing data lives
library(dplyr)
library(stringr)
library(foreach)
library(jsonlite)
library(purrr)
library(tidyr)
library(stringr)
library(readr)
data_directory_2020 <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/safegraph_data/social-distancing_v2/2020/'
processed_data_directory <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/'

In [ ]:
# Get the full set of months
months <- list.files(data_directory_2020)

# TODO: parallelize this so it is faster
# Cycle through the months
data_wrapped <- foreach(i= months, .combine='rbind') %do% {
    # construct the full directory for the month
    directory = paste0(c(data_directory_2020, i), sep='', collapse='')
    
    data_aggregate_month <- foreach(j = list.files(directory), .combine='rbind') %do% {
        # Only analyze dates for which we want to do our analysis (just to save some time)
        if(as.numeric(i) <= 3 | (as.numeric(i) == 4 & as.numeric(j) <= 19)) {
            # construct the full directory for the day, and the corresponding file containing the data
            subdirectory = paste0(c(directory, '/', j), sep='', collapse='')
            filename = paste0(c(subdirectory, '/', list.files(subdirectory)[1]), 
                              sep='', collapse='')
            # read in the data
            data = read_csv(filename,
                            col_types = cols(
                                origin_census_block_group = col_double(),
                                date_range_start = col_datetime(format = ""),
                                date_range_end = col_datetime(format = ""),
                                bucketed_distance_traveled = col_character(),
                                median_dwell_at_bucketed_distance_traveled = col_character(),
                                bucketed_home_dwell_time = col_character(),
                                at_home_by_each_hour = col_character(),
                                destination_cbgs = col_character(),
                                bucketed_away_from_home_time = col_character(),
                                bucketed_percentage_time_home = col_character()
                           ))
            # roll up the metrics to county level (instead of census tract level)
            data_aggregate <- data %>% 
            mutate(county_fips = floor(origin_census_block_group/10000000)) %>% 
            group_by(county_fips) %>% 
            summarise(device_count = sum(device_count),
                      completely_home_device_count = sum(completely_home_device_count),
                      part_time_work_behavior_devices = sum(part_time_work_behavior_devices),
                      full_time_work_behavior_devices = sum(full_time_work_behavior_devices)) %>% 
            # add a column that tells us what day this data comes from
            mutate(ds = paste0('2020-', i, '-', j))
          }
    }       
}

In [ ]:
# Write to file
write.csv(data_wrapped, file=paste0(c(processed_data_directory,
                                         'safegraph_social_distancing_aggregate_county.csv'),
                                       sep='',
                                       collapse=''),
          row.names=FALSE)